`model-training.ipynb` includes necessary steps for setting up a YOLO based model, gaining access to a Roboflow database and fine-tuning said model on said database.

The first section details the model training part. The second section acts as an emergency shutdown button. The user must stop the training loop, and run the commands in the second section to stop the training loop prematurely.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install -U ultralytics
!pip install roboflow

!pip uninstall -y wandb

In [ ]:
import os
import cv2
import math
import torch
import random
from ultralytics import YOLO

from roboflow import Roboflow
import matplotlib.pyplot as plt
from google.colab import userdata
from dotenv import load_dotenv, find_dotenv
from ultralytics.utils.plotting import plot_results

os.environ["WANDB_DISABLED"] = "True"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [ ]:
home = os.getcwd()

# Replace weight file with custom version or use default weights
model = YOLO("18042025.pt")

In [ ]:
!mkdir {home}/datasets
%cd {home}/datasets

_= load_dotenv(find_dotenv())
API_KEY = getpass.getpass("Please enter your API KEY: ")

rf = Roboflow(api_key = API_KEY)

# Workspace, project, and project version can all be modified
# To use a new, desired dataset
project = rf.workspace("basic-zf4d9").project("bubset-merged")
version = project.version(1)
dataset = version.download("yolov8")

In [ ]:
%cd {home}

yaml_path = f"{dataset.location}/data.yaml"

# 1. Training Loop

In [ ]:
# Other arguments that can be used can be found here:
# https://docs.ultralytics.com/usage/cfg/#train-settings

torch.cuda.empty_cache()
model.train(data = yaml_path,
            epochs = 1000,
            batch = 32,
            single_cls = True,
            amp = True,
            cache = True,
            save_period = 50,
            seed = 10,
            box = 5.0,
            cls = 1.25,
            freeze = 10,
            patience = 50,
            project = "/content/drive/MyDrive/YOLOExperiments",
            name = "18042025-based-run",
            augment = False,
            mosaic = False,
            plots = True
            )

# 2. In Case Training Needs to be Cut Short

Start by interrupting the execution of the training loop. Then run the cells underneath. First one evaluates the state of the model based on its best weights. The second one plots the results based on the `csv` file that is saved during the training loops.

```Be sure to double-check your directory paths, as they might be different from the ones below!```

In [ ]:
# Adapted from https://docs.ultralytics.com/modes/val/#example-validation-with-arguments

!yolo val model=/content/drive/MyDrive/YOLOExperiments/14042025-2/weights/best.pt \
         data={YAML_PATH} \
         imgsz=640 \
         save_json=True \
         plots=True

!mv /content/runs/detect/val /content/drive/MyDrive/YOLOExperiments/14042025-2/

In [ ]:
# Adapted from https://docs.ultralytics.com/reference/utils/plotting/#ultralytics.utils.plotting.plot_results

# Path to your experiment directory
log_dir = "/content/drive/MyDrive/YOLOExperiments/14042025-2"

# This will regenerate and save results.png inside log_dir
plot_results(file=log_dir + "/results.csv", dir=log_dir)